# Structure-Function Analysis
Written by J. Alexander Bae (jabae@princeton.edu)
- Related to Figure 6.

In [1]:
import numpy as np
import pandas as pd
import pickle

import datajoint as dj

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from scipy.sparse import csgraph
from meshparty import skeleton_io
from scipy.sparse.csgraph import shortest_path

import statsmodels.api as sm
from scipy.stats import pearsonr, percentileofscore

%matplotlib inline

/Users/jabae/.virtualenvs/jupyter/lib/python3.7/site-packages/python_jsonschema_objects/__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


## Functions

In [2]:
# Utils
def distance(x, y):
    
    return np.sqrt(np.sum((x-y)**2, axis=1))


def str2coord(str_coord, dtype="int"):
    
    l = str_coord[1:-1].split(" ")
    coord = []
    for j in range(len(l)):
        if l[j] != "":
            if dtype == "int": 
                coord.append(int(l[j]))
            elif dtype == "float":
                coord.append(float(l[j]))
    
    coord = np.array(coord)
    
    return coord

In [16]:
# DataJoint

# Get soma center coordinates ([nm])
def get_soma_loc(database, seg_id):
    
    key = {"segmentation": 185, "segment_id": seg_id}
    
    return (database.Soma() & key).fetch1("loc")/1000

In [3]:
# Plotting

def plot_trace_dir(ax, seg_id, angle, xlab="", ylab=""):
    
    scan_id = int(scan_list[pyc_func_list==seg_id])

    trace = RawTrace[seg_id]
    spike = Spike[seg_id]
    stimlab = StimulusLabel["scan{}".format(scan_id)]

    section_list = get_section(stimlab, angle)

    for s in section_list:
        n = s[1] - s[0] + 1
        plt.plot(np.arange(0,n*0.0674-0.001,0.0674), trace[s[0]:s[1]+1], linewidth=0.5)

    ax.set_xlim(0,1)
    ax.set_xticks(np.arange(0,1.1,0.5))
    ax.set_xticklabels(np.round(ax.get_xticks(),1), fontsize=15, fontname="Helvetica")
    ax.set_yticklabels(ax.get_yticks().astype("int"), fontsize=15, fontname="Helvetica")
    ax.set_xlabel(xlab, fontsize=18, fontname="Helvetica")
    ax.set_ylabel(ylab, fontsize=18, fontname="Helvetica")

## Connect to DataJoint database

- You need to be registered to access the database. Please refer to README in the repository for registration.

In [4]:
dj.config["database.host"] = "datajoint.ninai.org"
dj.conn()

pinky = dj.create_virtual_module("seung_pinky", "seung_pinky")

INFO:datajoint.settings:Setting database.host to datajoint.ninai.org


Please enter DataJoint username: jabae
Please enter DataJoint password: ········
Connecting jabae@datajoint.ninai.org:3306


## All pyramidal cells (PyCs)
- 363 pyramidal cells that have soma inside the volume.

In [6]:
pinky.Neuron()

segmentation,segment_id,manual_id
185,648518346349491311,466
185,648518346349492130,499
185,648518346349492197,503
185,648518346349492682,163
185,648518346349493472,549
185,648518346349493487,552
185,648518346349493874,-1
185,648518346349494004,47
185,648518346349494577,462
185,648518346349496405,174


In [10]:
key = {"segmentation": 185}
pyc_list = (pinky.Neuron() & key).fetch("segment_id")
n_pyc = pyc_list.shape[0]

## PyCs with functional data

In [12]:
cell_info_func = pd.read_csv("data/cell_functional_info.csv")

pyc_func_list = np.array(cell_info_func["segment_id"])
scan_list = np.array(cell_info_func["scan"]) + 1
n_func = pyc_func_list.shape[0]

osi_p = np.array(cell_info_func["osi_p"])
dsi_p = np.array(cell_info_func["dsi_p"])

tune_os_idx = osi_p<0.05
tune_ds_idx = dsi_p<0.05
tune_all_idx = tune_os_idx + tune_ds_idx

seg_id_os = pyc_func_list[tune_os_idx]
seg_id_ds = pyc_func_list[tune_ds_idx]
seg_id_tuned = pyc_func_list[tune_all_idx]

cell_info_func

,segment_id,scan,osi,osi_p,dsi,dsi_p
0,648518346349539895,1,1.399790,0.0085,0.360205,0.9077
1,648518346349537860,1,1.791797,0.0000,0.379977,0.7241
2,648518346349538440,1,1.369347,0.0025,0.257158,0.9983
3,648518346349538527,1,1.021334,0.0276,0.972555,0.9746
4,648518346349538209,1,1.863632,0.0000,0.100173,0.9763
...,...,...,...,...,...,...
107,648518346349537741,5,1.266736,0.0568,0.528219,0.9732
108,648518346349537901,5,1.163586,0.1148,1.162096,0.1404
109,648518346349538001,5,1.321359,0.0631,1.324058,0.0638
110,648518346349538251,5,1.436740,0.0139,1.399342,0.0287


## PyC - PyC subgraph
- Includes only pyramidal cells that have cell bodies in the volume.
- Connections among these pyramidal cells.

In [13]:
synapse_pyc_df = pd.DataFrame(data=pinky.PycSubgraph().fetch())
synapse_pyc_df

/Users/jabae/.virtualenvs/jupyter/lib/python3.7/site-packages/datajoint/connection.py:218: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


,segmentation,id,valid,pre_pt_position,pre_pt_supervoxel_id,pre_pt_root_id,ctr_pt_position,post_pt_position,post_pt_supervoxel_id,post_pt_root_id,size,spine_vol,exclude_conn
0,185,1484,1,"[91332, 57836, 1584]",97170491155950710,648518346349539437,"[91369, 57798, 1582]","[91332, 57748, 1584]",97170491155951077,648518346349531254,798,0.133004,0
1,185,2254,1,"[64762, 45414, 711]",89838917557174822,648518346349537978,"[64742, 45419, 708]","[64720, 45430, 708]",89838917557173796,648518346349537300,129,0.016346,0
2,185,3785,1,"[69646, 63086, 721]",91264984138384361,648518346349533058,"[69604, 63106, 721]","[69558, 63120, 721]",90983509161691227,648518346349538715,62,0.020969,0
3,185,3863,1,"[66776, 39946, 1138]",90396382837344883,648518346349539333,"[66750, 39982, 1139]","[66722, 39990, 1139]",90396382837366381,648518346349539806,62,0.018033,0
4,185,4062,1,"[89112, 48970, 2154]",96597662777759466,648518346349523975,"[89162, 48958, 2154]","[89198, 48936, 2154]",96597662777759456,648518346349537716,62,0.008805,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963,185,3546372,1,"[76660, 54312, 1086]",92945050790541263,648518346349539803,"[76699, 54305, 1090]","[76708, 54262, 1088]",92943951278915237,648518346349538239,130,0.044672,0
1964,185,3548009,1,"[94662, 61228, 453]",98018175966264919,648518346349531254,"[94712, 61234, 453]","[94728, 61228, 453]",98018175966265405,648518346349534360,61,0.023423,0
1965,185,3549101,1,"[110936, 59500, 1362]",102520706146780107,648518346349537716,"[110994, 59494, 1363]","[111032, 59498, 1362]",102520706146780799,648518346349539896,61,0.016120,0
1966,185,3549680,1,"[89680, 50944, 1475]",96599840326171296,648518346349539437,"[89706, 50906, 1474]","[89718, 50884, 1473]",96599840326170447,648518346349539781,61,0.021125,0


## Get soma center coordinates

In [15]:
pinky.Soma()

segmentation,segment_id,manual_id,loc
185,648518346349491311,466,=BLOB=
185,648518346349492130,499,=BLOB=
185,648518346349492197,503,=BLOB=
185,648518346349492682,163,=BLOB=
185,648518346349493472,549,=BLOB=
185,648518346349493487,552,=BLOB=
185,648518346349493874,-1,=BLOB=
185,648518346349494004,47,=BLOB=
185,648518346349494577,462,=BLOB=
185,648518346349496405,174,=BLOB=


#### All PyCs

In [18]:
pyc_soma_loc = np.zeros((n_pyc, 3))
for i in range(n_pyc):
    
    seg_id = pyc_list[i]
    pyc_soma_loc[i,:] = get_soma_loc(pinky, seg_id)

#### PyCs with functional data

In [19]:
pyc_func_soma_loc = np.zeros((n_func, 3))
for i in range(n_func):
    
    seg_id = pyc_func_list[i]
    idx = np.where(pyc_list==seg_id)[0][0]
    pyc_func_soma_loc[i,:] = pyc_soma_loc[idx,:]

## Compute dendrite length
- Unit: [$\mu$m]

#### All PyCs

In [22]:
d_skel = "data/smoothed_skeletons_v185/"

# Calculate dendrite length
dendrite_len = np.zeros((n_pyc,)) # In microns
for i in range(n_pyc):
    
    seg_id = pyc_list[i]
    seg_lab = np.load(d_skel+str(seg_id)+"_skeleton_label.npy")
    seg_skel = skeleton_io.read_skeleton_h5(d_skel+str(seg_id)+"_skeleton.h5")
    
    g = seg_skel.csgraph
    dend_idx = np.where((seg_lab==2)+(seg_lab==3)+(seg_lab==4))[0]
    g = g[dend_idx][:,dend_idx]
    
    dendrite_len[i] = np.sum(g)/1000

#### PyCs with functional data

In [23]:
dendrite_len_func = np.zeros(n_func)
for i in range(n_func):
    seg_id = pyc_func_list[i]
    dendrite_len_func[i] = dendrite_len[pyc_list==seg_id] 

## Compute in-connection density

In [25]:
pre_id_all = pinky.PycSubgraph().fetch("pre_pt_root_id")
post_id_all = pinky.PycSubgraph().fetch("post_pt_root_id")

real_synapse = np.zeros((pre_id_all.shape[0],2), dtype=np.uint64)
real_synapse[:,0] = pre_id_all
real_synapse[:,1] = post_id_all
real_connection = np.unique(real_synapse, axis=0)

## Compute total in-synapse density

In [ ]:
synapse_all_pyc_df = pd.read_csv("data/pyc_all_synapse.csv")
synapse_all_pyc_df

In [ ]:
post_id_all = np.array(synapse_all_pyc_df["post_pt_root_id"])
post_idx = np.where(np.isin(post_id_all, pyc_list))[0]

synapse_post_pyc_df = synapse_all_pyc_df.iloc[post_idx]

post_centroids = np.zeros((post_idx.shape[0],3))
for i in range(post_idx.shape[0]):
    post_centroids[i,:] = str2coord(synapse_post_pyc_df.iloc[i]["ctr_pt_position"])

post_id_list = np.array(synapse_post_pyc_df["post_pt_root_id"])
post_soma_loc = np.zeros(post_centroids.shape)
for i in range(post_id_list.shape[0]):  
    post_soma_loc[i,:] = pyc_soma_loc[pyc_list==post_id_list[i],:]
    
res = np.array([0.004, 0.004, 0.04])
post_centroids = post_centroids*res

# Filter out synapses within 15 um from the soma to remove perisomatic synapses.
d_post = distance(post_centroids, post_soma_loc)
valid = (d_post>15)

# All incoming synapses to pyramidal cells without perisomatic synapses.
synapse_post_pyc_df = synapse_post_pyc_df.iloc[valid]
synapse_post_pyc_dfb

In [ ]:
### Degree of cells with calcium signals
pre_id_all = pinky.PycSubgraph().fetch("pre_pt_root_id")
post_id_all = pinky.PycSubgraph().fetch("post_pt_root_id")
real_synapse_all = np.zeros((pre_id_all.shape[0],2), dtype=np.uint64)
real_synapse_all[:,0] = pre_id_all
real_synapse_all[:,1] = post_id_all

total_insyn_dens = np.zeros(n_func)
for i in range(n_func):
    
    seg_id = pyc_func_list[i]
    total_insyn_dens[i] = np.sum(real_synapse_all[:,1]==seg_id)/dendrite_len_func[i]